Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

In [2]:
?VB

search: 

A Variational Bayesian modeling approach. Models are defined by  bipartite graphs in which factors defining terms in the variational objective are connected to nodes defining variables.


VB VBModel A_rdiv_Bt A_rdiv_Bc A_ldiv_Bt A_ldiv_Bc A_ldiv_B! At_rdiv_B



Let's try to make a Gaussian node with several different sizes and parameterizations:

In [3]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

3

In [4]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [5]:
μ[1]

3-element Array{Float64,1}:
 0.720048
 0.314559
 0.652488

In [6]:
Σ[1]

3x3 Array{Float64,2}:
 0.136961  0.35471   0.165383
 0.35471   1.43069   0.452166
 0.165383  0.452166  0.478905

In [7]:
det(Σ[2])

0.02934456965698479

In [8]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...)

5x5 Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}:
 FullNormal(
dim: 3
μ: [0.7200481158970189,0.31455935400122126,0.6524880003299676]
Σ: 3x3 Array{Float64,2}:
 0.136961  0.35471   0.165383
 0.35471   1.43069   0.452166
 0.165383  0.452166  0.478905
)
   …  FullNormal(
dim: 3
μ: [0.6272607783132158,0.7759185944110354,0.003611624194325147]
Σ: 3x3 Array{Float64,2}:
 0.885208  0.977873  0.457397
 0.977873  1.12024   0.39697 
 0.457397  0.39697   0.541515
)

 FullNormal(
dim: 3
μ: [0.4090587615519383,0.01872949414367464,0.22755534539464395]
Σ: 3x3 Array{Float64,2}:
 1.1205    0.804299  0.646437
 0.804299  1.14864   1.05553 
 0.646437  1.05553   1.0312  
)
     FullNormal(
dim: 3
μ: [0.029704938052093288,0.7406146263203168,0.845196619300917]
Σ: 3x3 Array{Float64,2}:
 1.17715   0.479757  0.960111
 0.479757  0.203115  0.383751
 0.960111  0.383751  1.57785 
)
 
 FullNormal(
dim: 3
μ: [0.5449660109946102,0.38426214806573045,0.2599979564991193]
Σ: 3x3 Array{Float64,2}:
 0.850382 

In [9]:
rr = RandomNode(aa, Factor[]);

In [10]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [11]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [12]:
ss = RandomNode(MultivariateNormal, pars...);

In [13]:
E(ss)

5x5 Array{Array{Float64,1},2}:
 [0.7200481158970189,0.31455935400122126,0.6524880003299676]   …  [0.6272607783132158,0.7759185944110354,0.003611624194325147]
 [0.4090587615519383,0.01872949414367464,0.22755534539464395]     [0.029704938052093288,0.7406146263203168,0.845196619300917] 
 [0.5449660109946102,0.38426214806573045,0.2599979564991193]      [0.9037670888473266,0.6283776964154486,0.9756615261026564]  
 [0.47634021975956276,0.9862431891887227,0.4595751593567998]      [0.5182185262709909,0.5845869068389591,0.4827343934347623]  
 [0.5792039948690257,0.045380698682431,0.3790780181248168]        [0.553813874754997,0.07656472504329326,0.4105883266909225]  

In [14]:
?E

search: 

Calculates the expected value of a Node x.


In [15]:
typeof(ss).parameters[1]

Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}}

# Make a factor and check its value

In [16]:
μ = rand(10)
τ = rand(10)
x = RandomNode(Normal, rand(10), rand(10))
f = LogNormalFactor(x, μ, τ)

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Union{VB.ConstantNode{T<:Number,N},VB.RandomNode{D<:Distributions.Distribution{F<:Distributions.VariateForm,S<:Distributions.ValueSupport},N}}}, ::Array{Float64,1})
This may have arisen from a call to the constructor Union{VB.ConstantNode{T<:Number,N},VB.RandomNode{D<:Distributions.Distribution{F<:Distributions.VariateForm,S<:Distributions.ValueSupport},N}}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[16], in expression starting on line 4

In [17]:
convert(ConstantNode, μ)

VB.ConstantNode{Float64,1}([0.42971973365110383,0.03166049107083846,0.6925697375152744,0.719932209970078,0.20650517194051732,0.9386185743587601,0.2879769780446111,0.7197382089444124,0.5887029181557208,0.253605921938421],VB.Factor[])

E e eu ENV end Expr eye exp eta esc erf eps eof eig Enum Elog else expm

